#Task 1

In [2]:
import re
pattern1 = re.compile(r'abc')
result1 = pattern1.match('abcdef')
if result1:
  print("Match found:", result1.group())

pattern2 = re.compile(r'.')
result2 = pattern2.search('Hello')
if result2:
  print("Character found:", result2.group())

pattern3 = re.compile(r'[aeiou]')
result3 = pattern3.search('Hello')
if result3:
  print("Vowel found:", result3.group())

pattern4 = re.compile(r'\d{3}-\d{2}-\d{4}')
result4 = pattern4.match('123-45-6789')
if result4:
  print("Social Security Number:", result4.group())

pattern5 = re.compile(r'\.')
result5 = pattern5.search('www.example.com')
if result5:
  print("Dot found:", result5.group())

pattern6 = re.compile(r'(\d{2})/(\d{2})/(\d{4})')
result6 = pattern6.match('23/06/2025')
if result6:
  print("Day:", result6.group(1))
  print("Month:", result6.group(2))
  print("Year:", result6.group(3))

pattern7 = re.compile(r'cat|dog')
result7 = pattern7.search('I have a cat and a dog')
if result7:
  print("Animal found:", result7.group())

Match found: abc
Character found: H
Vowel found: e
Social Security Number: 123-45-6789
Dot found: .
Day: 23
Month: 06
Year: 2025
Animal found: cat


#Task 2

In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

text = "Tokenization without trnsformer is Straightforward with tools like NLTK"
tokens = word_tokenize(text)
print("Tokens:",tokens);

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
text = "Tokenization without transformers is Straightforward with tools like NLTK"
tokens_transformers = tokenizer(text,return_tensors="pt")
print("Tokens Transformer:",tokens_transformers);

tokens_transformers_list = tokenizer.convert_ids_to_tokens(tokens_transformers["input_ids"][0].numpy().tolist());
print("Transformers token(list):",tokens_transformers_list);

decode_text = tokenizer.decode(tokens_transformers['input_ids'][0],skip_special_tokens=True)
print("Decode Text:", decode_text);

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokens: ['Tokenization', 'without', 'trnsformer', 'is', 'Straightforward', 'with', 'tools', 'like', 'NLTK']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokens Transformer: {'input_ids': tensor([[  101, 19204,  3989,  2302, 19081,  2003, 19647,  2007,  5906,  2066,
         17953,  2102,  2243,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Transformers token(list): ['[CLS]', 'token', '##ization', 'without', 'transformers', 'is', 'straightforward', 'with', 'tools', 'like', 'nl', '##t', '##k', '[SEP]']
Decode Text: tokenization without transformers is straightforward with tools like nltk


#Task 3

In [8]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
import spacy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('punkt_tab')
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

corpus = "One disadvantage of using 'Best Of' samping is that it may lead to limited exploration of the model's knowledge and creativity. By focusing on the most probable next words, the model might generate responses that are safe and conventional, potentially missing out on more diverse and innovative outputs. The lack of exploration could result in repetitive or less imaginative responses, especially in situations where novel and unconventional ideas are desired.To address this limitation, other sampling strategies like temperature-based sampling or top-p (nucleus) sampling can be employed to introduce more randomness and encourage the model to explore a broader range of possibilities. However, it's essential to carefully balance exploration and exploitation based on the specific requirements of the task or application."

tokens = word_tokenize(corpus)
lemmatized_tokens = [token.lemma_ for token in nlp(corpus)]
all_tokens = tokens + lemmatized_tokens

tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_tokens)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in all_tokens:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_length = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = np.array(y)

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_length-1))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.0000e+00 - loss: 4.6717
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.3333 - loss: 4.6634
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 1.0000 - loss: 4.6551
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 1.0000 - loss: 4.6467
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 1.0000 - loss: 4.6382
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 1.0000 - loss: 4.6294
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 1.0000 - loss: 4.6205
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 4.6113
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 1.0000 - loss: 4.6017
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 1.0000 - loss: 4.5918


#Task 4

In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

def load_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def tokenize_document(document):
    tokens = word_tokenize(document)
    return [word.lower() for word in tokens if word.isalpha()]

def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]

def find_morphology(tokens):
    fdist = FreqDist(tokens)
    return fdist.most_common(10)

document_path = '/content/tourism_documents.txt'

document = load_document(document_path)
tokens = tokenize_document(document)
tokens_without_stopwords = remove_stopwords(tokens)
morphology = find_morphology(tokens_without_stopwords)

print("Morphology of the document:")
for word, frequency in morphology:
    print(f"{word}: {frequency}")

Morphology of the document:
traveling: 1
abroad: 1
make: 1
sure: 1
prepare: 1
necessary: 1
including: 1
passport: 1
visa: 1
travel: 1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Task 5

In [4]:
import string
import random
import nltk
from nltk.corpus import stopwords, reuters
from collections import Counter, defaultdict
from nltk import FreqDist, ngrams
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('reuters')
sents = reuters.sents()
stop_word = set(stopwords.words('english'))
string.punctuation = string.punctuation + ' " ' + ' " ' + ' - ' + ' _ '
removal_list = list(stop_word) + list(string.punctuation) + ['\t', 'rt']
unigram = []
bigram = []
trigram = []
tokenized_text = []
for sentence in sents:
    sentence = list(map(lambda x: x.lower(), sentence))
    for word in sentence:
        if word == '.':
            sentence.remove(word)
        else:
            unigram.append(word)
            tokenized_text.append(word)
    bigram.extend(list(ngrams(sentence, 2, pad_left=True, pad_right=True)))
    trigram.extend(list(ngrams(sentence, 3, pad_left=True, pad_right=True)))
def remove_stopwords(x):
    y = []
    for item in x:
        if isinstance(item, tuple):
            count = 0
            for word in item:
                if word not in removal_list:
                    count = 1
                    break
            if count == 1:
                y.append(item)
        else:
            if item not in removal_list:
                y.append(item)
    return y

unigram = remove_stopwords(unigram)
bigram = remove_stopwords(bigram)
trigram = remove_stopwords(trigram)
freq_uni = FreqDist(unigram)
freq_bi = FreqDist(bigram)
freq_tri = FreqDist(trigram)
d = defaultdict(Counter)
for a, b, c in freq_tri:
    if (a is not None) and (b is not None) and (c is not None):
        d[a, b][c] += freq_tri[a, b, c]
def pick_word(counter):
    "choose a random element"
    return random.choice(list(counter.elements()))
prefix = "he", "is"
print(" ".join(prefix))
s = " ".join(prefix)
for i in range(19):
    if prefix in d:
        suffix = pick_word(d[prefix])
        s = s + ' ' + suffix
        print(s)
        prefix = prefix[1], suffix
    else:
        print(f"Prefix {prefix} not found in dictionary. Stopping text generation.")
        break

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


he is
he is one
he is one of
he is one of which
he is one of which analysts
he is one of which analysts estimate
he is one of which analysts estimate phillips
he is one of which analysts estimate phillips "
he is one of which analysts estimate phillips " cash
he is one of which analysts estimate phillips " cash investment
Prefix ('cash', 'investment') not found in dictionary. Stopping text generation.


#Task 6

In [5]:
from nltk.util import ngrams
from nltk.lm import Laplace
from nltk.tokenize import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
import nltk

nltk.download('punkt_tab')

def ngram_smoothing(sentence, n):
  tokens = word_tokenize(sentence.lower())
  train_data, padded_sents = padded_everygram_pipeline(n, tokens)
  model = Laplace(n)
  model.fit(train_data, padded_sents)
  return model

sentence = "his is the sample example taken to evaluate the n-gram smoothening technique"
print("Enter a sentence: ",sentence)
n = 10
print("Enter the value of N for N-grams:",n)
model = ngram_smoothing(sentence, n)
context = tuple(sentence.lower().split()[-n+1:])
next_words = model.generate(3, text_seed=context)
print("Next words:", ' '.join(next_words))

Enter a sentence:  his is the sample example taken to evaluate the n-gram smoothening technique
Enter the value of N for N-grams: 10
Next words: t e c


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


#Task  7

In [3]:
import nltk
nltk.download("all");
from nltk.tag import HiddenMarkovModelTagger
from nltk.corpus import treebank
nltk.download('treebank')
corpus = treebank.tagged_sents()
train_data = corpus[:3000]
test_data = corpus[3000:]
hmm_tagger = HiddenMarkovModelTagger.train(train_data)
sentance = "I Love NLP"
tokens = nltk.word_tokenize(sentance)
tagged_sentance = hmm_tagger.tag(tokens)
print(tagged_sentance)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

[('I', 'PRP'), ('Love', 'MD'), ('NLP', 'VB')]
